## Imports

In [1]:
import logging
import random
from functools import reduce
from typing import Callable

logging.getLogger().setLevel(logging.INFO)

## Define the class Nim

In [2]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2+1 for i in range(num_rows)]
        self._k = k

    def nimming_remove(self, row: int, num_objects: int) -> None:
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects < self._k
        self._rows[row] -= num_objects
    
    def nimming_add(self, row: int, num_objects: int) -> None:
        self._rows[row] += num_objects

    def goal(self) -> bool:
        # Check if someone has won
        return sum(self._rows) == 0

## Random player

In [3]:
class RandomPlayer:
    def __init__(self) -> None:
        self._moves = []
        self._nMoves = 0
    
    def play(self, nim: Nim) -> None:
        # Chose a random row and a random number of pieces
        x = random.randint(0, len(nim._rows)-1)
        while nim._rows[x] == 0:
            x = random.randint(0, len(nim._rows)-1)
        y = random.randint(1, nim._rows[x])
        # Update the attributes
        self._nMoves += 1
        self._moves.append(f"{y} items from row {x}")
        nim.nimming_remove(x, y)
        
    def printSolution(self) -> None:
        logging.info(f" Random player won, moves have been: {self._moves}; total: {self._nMoves}")

## Nim sum verifier

In [4]:
def verify_state(nim: Nim) -> bool:
    # Verify of the new game state is useful for the player with the nim sum rule
    xor = reduce(lambda a, b: a ^ b, nim._rows)
    if xor == 0:
        return True
    else:
        return False

## Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)

### Expert System

In [5]:
class ExpertSystem:
    def __init__(self) -> None:
        self._nMoves = 0
        self._moves = []

    def printSolution(self) -> None:
        logging.info(f" Expert system won, moves have been: {self._moves}; total: {self._nMoves}")
    
    def play(self, nim: Nim) -> None:
        possible = [(r, o) for r, c in enumerate(nim._rows) for o in range(1, c + 1)]
        for ply in possible:
            nim.nimming_remove(ply[0], ply[1])
            valid = verify_state(nim)
            if valid:
                break
            else:
                nim.nimming_add(ply[0], ply[1])

        if not valid:
            ply = random.choice(possible)
            nim.nimming_remove(ply[0], ply[1])
            
        self._nMoves += 1
        self._moves.append(f"{ply[1]} items from row {ply[0]}")

## Task3.2: An agent using evolved rules

In [ ]:
def cook_status(nim: Nim) -> dict:
    cooked = dict()
    cooked['possible_moves'] = [(r, o) for r, c in enumerate(nim._rows) for o in range(1, c + 1) if nim._k is None or o <= nim._k]
    cooked['active_rows_number'] = sum(r > 0 for r in nim._rows)
    cooked['shortest_row'] = min((x for x in enumerate(nim._rows) if x[1] > 0), key= lambda y: y[1])[0]
    cooked['longest_row'] = max((x for x in enumerate(nim._rows) if x[1] > 0), key= lambda y: y[1])[0]
    cooked['nim_status'] = verify_state(nim)

    brute_force = list()
    for m in cooked['possible_moves']:
        nim.nimming_remove()
    return cooked

In [ ]:
def optimal_strategy(nim: Nim) -> tuple:
    data = cook_status(nim)
    return next(bf for bf in data['brute_force'] if bf[1] == 0)[0]

In [14]:
def evaluate():
    pass

def make_strategy(genome: dict) -> Callable:
    def evolvable(nim: Nim) -> tuple:
        data = cook_status(nim)
        genome

        if random.random() < genome['p']:
            ply = (data['shortest_row'], random.randint(1, nim._rows[data['shortest_row']]))
        else:
            ply = (data['longest_row'], random.randint(1, nim._rows[data['longest_row']]))
        
        return ply
    return evolvable

SyntaxError: incomplete input (3588321753.py, line 4)

## Task3.3: An agent using minmax

### MinMax System

In [6]:
class MinMaxSystem:
    def __init__(self) -> None:
        self._nmoves = 0
        self._moves = []

    def printSolution(self) -> None:
        logging.info(f" MinMax system won, moves have been: {self._moves}; total: {self._nmoves}")

    def minmax(self, nim: Nim, player: bool, step: int):
        possible = [(r, o) for r, c in enumerate(nim._rows) for o in range(1, c + 1)]
        # True player is the MinMax system, False player is the other player
        if not possible:
            if player:
                # No win -> the other player arrives with the table empty
                return (None, -1, step)
            else:
                # Win -> the MinMax system arrives with the table empty
                return (None, 1, step)
        evaluations = list()
        for ply in possible:
            nim.nimming_remove(ply[0], ply[1])
            # Recursive call
            _, val, weight = self.minmax(nim, not player, step+1)
            if val == 1 and step != 0:
                # Found a good path, return it
                nim.nimming_add(ply[0], ply[1])
                return (ply, val, weight)
            elif val == 1 and step == 0:
                # Found a good path, save it
                nim.nimming_add(ply[0], ply[1])
                evaluations.append((ply, val, weight))
            else:
                nim.nimming_add(ply[0], ply[1])
                evaluations.append((ply, val, weight))
        return max(evaluations, key=lambda x: -x[2])

    def play(self, nim: Nim):
        best_ply, _, _ = self.minmax(nim, player=True, step=0)
        nim.nimming_remove(best_ply[0], best_ply[1])
        self._nmoves += 1
        self._moves.append(f"{best_ply[1]} items from row {best_ply[0]}")

## Task3.4: An agent using reinforcement learning

## Games

### Random vs Expert

In [ ]:
nim = Nim(7)
player1 = RandomPlayer()
player2 = ExpertSystem()
win = random.choice([1, 2])
print("Start game: ", nim._rows)
while nim.goal() == False:
    if win == 1:
        player1.play(nim)
        print("Game after Random player played: ", nim._rows)
        win = 2
    else:
        player2.play(nim)
        print("Game after Expert system played: ", nim._rows)
        win = 1
if win == 1:
    player2.printSolution()
else:
    player1.printSolution()

### Random vs MinMax

In [7]:
nim = Nim(7)
player1 = RandomPlayer()
player2 = MinMaxSystem()
win = random.choice([1, 2])
print("Start game: ", nim._rows)
while nim.goal() == False:
    if win == 1:
        player1.play(nim)
        print("Game after Random player played: ", nim._rows)
        win = 2
    else:
        player2.play(nim)
        print("Game after MinMax system played: ", nim._rows)
        win = 1
if win == 1:
    player2.printSolution()
else:
    player1.printSolution()

Start game:  [1, 3, 5, 7, 9, 11, 13]
Game after Random player played:  [1, 3, 5, 7, 0, 11, 13]


ValueError: too many values to unpack (expected 2)

### Expert vs MinMax

In [ ]:
nim = Nim(7)
player1 = ExpertSystem()
player2 = MinMaxSystem()
win = random.choice([1, 2])
print("Start game: ", nim._rows)
while nim.goal() == False:
    if win == 1:
        player1.play(nim)
        print("Game after Expert System played: ", nim._rows)
        win = 2
    else:
        player2.play(nim)
        print("Game after MinMax system played: ", nim._rows)
        win = 1
if win == 1:
    player2.printSolution()
else:
    player1.printSolution()